# Fine-Tuned

This notebook extracts respiration signals for each subject using the fine-tuned models.

In [ ]:
import os.path

import respiration.utils as utils

# Map model names to their paths
models = {
    'BP4D_PseudoLabel_EfficientPhys': utils.file_path('data', 'rPPG-Toolbox', 'BP4D_PseudoLabel_EfficientPhys.pth'),
    'MA-UBFC_efficientphys': utils.file_path('data', 'rPPG-Toolbox', 'MA-UBFC_efficientphys.pth'),
    'PURE_EfficientPhys': utils.file_path('data', 'rPPG-Toolbox', 'PURE_EfficientPhys.pth'),
    'SCAMPS_EfficientPhys': utils.file_path('data', 'rPPG-Toolbox', 'SCAMPS_EfficientPhys.pth'),
    'UBFC-rPPG_EfficientPhys': utils.file_path('data', 'rPPG-Toolbox', 'UBFC-rPPG_EfficientPhys.pth'),
}

manifests = []

signals_dir = utils.dir_path('outputs', 'signals', mkdir=True)

# Store the manifests
manifests_path = utils.join_paths(signals_dir, 'efficient_phys_manifest.json')
utils.write_json(manifests_path, manifests)

utils.pretty_print(models)

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()
scenarios = dataset.get_scenarios(['101_natural_lighting'])

device = utils.get_torch_device()
dim = 72
frame_depth = 20

In [ ]:
import torch
import pandas as pd
import datetime as dt

from tqdm.auto import tqdm
from respiration.extractor.efficient_phys import EfficientPhys

predictions = []

for (subject, setting) in tqdm(scenarios):
    print(f"Processing {subject} - {setting}")

    video_path = dataset.get_video_path(subject, setting)
    frame_count = utils.get_frame_count(video_path)
    chunk_size = (frame_count // frame_depth) * frame_depth - (frame_depth - 1)
    # chunk_size = frame_depth * 20 + 1

    frames, meta = utils.read_video_rgb(video_path, chunk_size)
    frames = utils.down_sample_video(frames, dim)
    frames = torch.tensor(frames, dtype=torch.float32, device=device).permute(0, 3, 1, 2)

    for (model_id, model_path) in models.items():
        print(f"--> Using {model_id} model")
        # Wrap modul in nn.DataParallel to fix the model loading issue
        model = torch.nn.DataParallel(EfficientPhys(img_size=dim, frame_depth=frame_depth))
        model.to(device)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval()

        start = dt.datetime.now()

        with torch.no_grad():
            prediction = model(frames).cpu().detach().numpy().squeeze()

        predictions.append({
            'model': model_id,
            'subject': subject,
            'setting': setting,
            'duration': dt.datetime.now() - start,
            'chunk_size': chunk_size,
            'sampling_rate': meta.fps,
            'signal': prediction.tolist(),
        })

    del frames

predictions = pd.DataFrame(predictions)

# Store the predictions to csv
signals_path = utils.join_paths(signals_dir, 'efficient_phys_predictions.csv')

# Append the predictions to the csv file
if os.path.exists(signals_path):
    predictions.to_csv(signals_path, mode='a', header=False, index=False)
else:
    predictions.to_csv(signals_path, index=False)

In [ ]:
predictions.head()